In [77]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Run

In [78]:
import pandas as pd

runs = [
    "VAL_naive_doctaet-simple_zs-mistralaiMistral7BInstructv03-20240611-095123",
    "VAL_naive_doctaet-simple_fs-mistralaiMistral7BInstructv03-20240611-102154",
    "VAL_naive_doctaet-simple_zs-googlegemma7bit-20240611-130659",
    "VAL_naive_doctaet-simple_fs-googlegemma7bit-20240611-133609",
    "VAL_llama3_70b_fewshot_optimized02-20240531-105142",
    "VAL_llama3_8b_few_shot_template_initial-20240604-125136",
    "VAL_naive_doctaet-simple_zs-llama38b-20240611-212642",
    "VAL_naive_doctaet-simple_fs-llama38b-20240611-213146",
    "VAL_naive_doctaet-simple_zs-gemma7b-20240612-073229",
    "VAL_naive_doctaet-simple_fs-gemma7b-20240612-073907",
    "VAL_naive_doctaet-simple_zs-mistral7b-20240612-074427",
    "VAL_naive_doctaet-simple_fs-mistral7b-20240612-075256",
    "VAL_naive_doctaet-simple_zs-llama370b-20240612-083003",
    "VAL_naive_doctaet-simple_fs-llama370b-20240612-085839",
    
]

# Top runs on Test:
# - TEST_llama3_70b_fewshot_initial-20240601-211554
dfs = []

for run in runs:
    path = f"results/{run}/df.feather"

    df = pd.read_feather(path)
    try:
        df = df.rename({"annontation": "annotation"}, axis=1) # typo in early versions
    except:
        pass
    dfs.append(df)


# Post Processing

In [79]:
from src.content_extraction import format

# Some older runs might not be in the right format -> format function is idempotent
for df in dfs:
    df["annotation"] = df["annotation"].apply(format)
    df["annotation"] = df["annotation"].astype(pd.StringDtype())
    df["ground_truth"] = df["ground_truth"].astype(pd.StringDtype())

In [80]:
df.dtypes

run              object
f                object
annotation       string
ground_truth     string
inference_s     float64
dtype: object

# Evaluation

In [81]:
# evaluate
from scoring_program.evaluation import Metrics

results = []
for df in dfs:
    result = Metrics.evaluate_property_wise_json_based(
        label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"])
    )

    result.update(
        Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
    )
    result["run"] = df["run"][0]
    result["score"] = result["partial_f1s_overall"]
    result["num_samples"] = len(df)
    results.append(pd.DataFrame([result]))

df_res = pd.concat(results)
df_res

,general_accuracy,exact_recalls_task,exact_recalls_dataset,exact_recalls_metric,exact_recalls_Score,exact_recalls_overall,partial_recalls_task,partial_recalls_dataset,partial_recalls_metric,partial_recalls_Score,...,partial_f1s_metric,partial_f1s_Score,partial_f1s_overall,rouge1,rouge2,rougeL,rougeLsum,run,score,num_samples
0,51.0,1.48,1.48,5.42,1.48,2.46,6.90,2.46,6.40,1.46,...,6.52,2.01,4.58,17.92,10.79,16.93,16.62,VAL_naive_doctaet-simple_zs-mistralaiMistral7B...,4.58,100
0,64.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,33.94,13.41,33.02,32.97,VAL_naive_doctaet-simple_fs-mistralaiMistral7B...,0.00,100
0,51.0,0.00,0.99,0.00,0.00,0.25,0.49,1.48,0.00,0.00,...,0.00,0.00,0.58,14.18,2.69,12.26,12.13,VAL_naive_doctaet-simple_zs-googlegemma7bit-20...,0.58,100
0,58.0,0.00,3.45,2.46,0.99,1.72,0.49,3.45,2.96,1.46,...,2.82,1.44,2.01,18.75,3.34,18.05,18.02,VAL_naive_doctaet-simple_fs-googlegemma7bit-20...,2.01,100
0,49.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,49.00,0.00,49.00,49.00,VAL_llama3_70b_fewshot_optimized02-20240531-10...,0.00,100
0,69.0,0.49,0.00,0.00,0.00,0.12,0.49,0.00,0.00,0.00,...,0.00,0.00,0.24,33.78,7.35,32.73,32.64,VAL_llama3_8b_few_shot_template_initial-202406...,0.24,100
0,64.0,0.00,2.96,2.46,0.00,1.35,0.00,2.96,2.46,0.00,...,3.22,0.00,1.75,26.66,5.30,25.40,25.57,VAL_naive_doctaet-simple_zs-llama38b-20240611-...,1.75,100
0,63.0,0.49,0.99,1.48,0.00,0.74,1.48,1.97,2.96,0.49,...,4.05,0.72,2.32,24.20,4.39,23.06,23.11,VAL_naive_doctaet-simple_fs-llama38b-20240611-...,2.32,100
0,51.0,0.00,0.00,0.00,1.97,0.49,0.00,0.00,0.00,1.94,...,0.00,3.76,0.94,7.72,1.74,7.22,7.12,VAL_naive_doctaet-simple_zs-gemma7b-20240612-0...,0.94,100
0,54.0,0.00,0.49,1.97,1.97,1.11,0.00,0.49,1.97,1.94,...,3.86,3.49,2.13,13.78,1.13,13.32,13.08,VAL_naive_doctaet-simple_fs-gemma7b-20240612-0...,2.13,100


In [82]:
df = dfs[-1]
df

,run,f,annotation,ground_truth,inference_s
0,VAL_naive_doctaet-simple_fs-llama370b-20240612...,1503.05062,unanswerable,unanswerable,4.612794
1,VAL_naive_doctaet-simple_fs-llama370b-20240612...,1109.0784,unanswerable,unanswerable,2.107736
2,VAL_naive_doctaet-simple_fs-llama370b-20240612...,2210.16422v1,[{'LEADERBOARD':{'Task': 'Scientific Paper Sum...,[{'LEADERBOARD': {'Task': 'Text Summarization'...,25.085012
3,VAL_naive_doctaet-simple_fs-llama370b-20240612...,2310.07488v2,"[{'LEADERBOARD':{'Task': 'GSM8K', 'Metrics': [...",[{'LEADERBOARD': {'Task': 'Arithmetic Reasonin...,25.520247
4,VAL_naive_doctaet-simple_fs-llama370b-20240612...,2210.15425v1,"['Keyword Spotting'], 'Datasets': ['ND-API'], ...","[{'LEADERBOARD': {'Task': 'Keyword Spotting', ...",13.434302
...,...,...,...,...,...
95,VAL_naive_doctaet-simple_fs-llama370b-20240612...,2006.09264v3,unanswerable,[{'LEADERBOARD': {'Task': 'Neural Architecture...,2.175533
96,VAL_naive_doctaet-simple_fs-llama370b-20240612...,1304.2639,unanswerable,unanswerable,1.269445
97,VAL_naive_doctaet-simple_fs-llama370b-20240612...,1803.10683v3,[{'LEADERBOARD':{'Task': 'Pose-based Alignment...,"[{'LEADERBOARD': {'Task': 'Pose Estimation', '...",21.611958
98,VAL_naive_doctaet-simple_fs-llama370b-20240612...,2006.06936v2,[Here is the extracted information:**Tasks:** ...,[{'LEADERBOARD': {'Task': 'Neural Architecture...,12.229535
